In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
tmdb = pd.read_csv('./data/tmdb-movies.csv', encoding='raw_unicode_escape')

This data set contains information about 10,000 movies collected from The Movie Database (TMDb), including user ratings and revenue.

- Certain columns, like ‘cast’ and ‘genres’, contain multiple values separated by pipe (|) characters.
- There are some odd characters in the ‘cast’ column. Don’t worry about cleaning them. You can leave them as is.
- The final two columns ending with “_adj” show the budget and revenue of the associated movie in terms of 2010 dollars, accounting for inflation over time.

In [3]:
def create_overview(df):
    # Function to create an aggregates overview from existing pandas methods for reuse
    # Use 'describe' and flip it for better readability
    overview = df.describe(include='all').T
    # Integrate dtypes
    overview['dtype'] = df.dtypes
    # Integrate an example (1st row)
    overview['example'] = df.head(1).T
    # 
    overview['complete_%'] = 100 * overview['count'] / df.shape[0]
    # Rename median
    overview.rename(index=str, columns={'50%': 'median'}, inplace=True)
    # Create new features (that, in 'describe', are normally only included with object columns)
    overview['top_alt'] = ''
    overview['freq_alt'] = 0
    overview['unique_alt'] = 0
    # Calculate new features
    for col in df.columns:
        value_counts = df[col].value_counts()
        overview.loc[col, 'top_alt'] = str(value_counts.index[0])
        overview.loc[col, 'freq_alt'] = value_counts.iloc[0]
        overview.loc[col, 'unique_alt'] = df[col].drop_duplicates().count()
    # Reorganize the overview (whilst neglecting some columns from 'describe')
    overview = overview[[
        'example', 
        'dtype',
        'count',
        'complete_%',
        'unique_alt',
        'top_alt',
        'freq_alt',
        'mean',
        'std',
        'min',
        'median',
        'max'
        ]]
    return overview

create_overview(tmdb)

,example,dtype,count,complete_%,unique_alt,top_alt,freq_alt,mean,std,min,median,max
id,135397,int64,10866,100,10865,42194,2,66064.2,92130.1,5,20669,417859
imdb_id,tt0369610,object,10856,99.908,10855,tt0411951,2,NaN,NaN,NaN,NaN,NaN
popularity,32.9858,float64,10866,100,10814,0.109305,2,0.646441,1.00018,6.5e-05,0.383856,32.9858
budget,150000000,int64,10866,100,557,0,5696,1.46257e+07,3.09132e+07,0,0,4.25e+08
revenue,1513528810,int64,10866,100,4702,0,6016,3.98233e+07,1.17003e+08,0,0,2.78151e+09
original_title,Jurassic World,object,10866,100,10571,Hamlet,4,NaN,NaN,NaN,NaN,NaN
cast,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,object,10790,99.3006,10719,Louis C.K.,6,NaN,NaN,NaN,NaN,NaN
homepage,http://www.jurassicworld.com/,object,2936,27.0201,2896,http://www.missionimpossible.com/,4,NaN,NaN,NaN,NaN,NaN
director,Colin Trevorrow,object,10822,99.5951,5067,Woody Allen,45,NaN,NaN,NaN,NaN,NaN
tagline,The park is open.,object,8042,74.0107,7997,Based on a true story.,5,NaN,NaN,NaN,NaN,NaN


Probably, this database has one duplicate row (max `count` = max `unique_alt` + 1 = 10.865).

However, before we dig a little deeper into the various features, it's useful to define the main purpose of the following analysis. This allows to focus on the features (hopefully) more relevant to this purpose.  

There are two aspects, we analyze this dataset here, whilst **focussing on "goodness" in terms of `'vote_average'`**
- How do "good movies" perform "economically" and how did this change over time?
- What are the artistic "ingredients" of a "good movie"? Especially, which genres and artists are the most "sucessful" ones?

Now, let's describe the fields, decide if they are being kept for the purposes of this special analysis, and describe, which issues are to be handled with in the data cleaning:  


| Field | Understanding & Assumptions | Keeper? | Issues |  
|:--- |:---|:---:|:---|  
| `'id'`  | Serves as an identifier | Yes | Convert to `str`, otherwise probably none |  
| `'imdb_id'` | Serves as an identifier | **No** | We take `'id'` instead | 
| `'popularity'` | Popularity rating for TMDB's website purposes (s. [here](https://developers.themoviedb.org/3/getting-started/popularity)) | **No** | Neglecting this feature | 
| `'budget'` | Film's budget in USD | Yes | Be aware of `0` values | 
| `'revenue'` | Film's revenue in USD | Yes | Be aware of `0` values | 
| `'original_title'` | Film title | Yes | Be aware of duplicates | 
| `'cast'` | Name of actors | Yes | Multiple items in a field and missing values | 
| `'homepage'` | Url | **No** | Neglecting this feature | 
| `'director'` | Name of director | Yes | Missing values | 
| `'tagline'` | Short description of film | **No** | Neglecting this feature, using `keywords` instead | 
| `'keywords'` | Tags for film description | Yes | Multiple items in a field and missing values | 
| `'overview'` | Longer description of film | **No** | Neglecting this feature, using `keywords` instead | 
| `'runtime'` | Runtime in minutes | Yes | Beware of `0` values | 
| `'genres'` | Tags for genre of film | Yes | Multiple items in a field and missing values | 
| `'production_companies'` | Name of production companies | Yes | Multiple items in a field and missing values | 
| `'release_date'` | Date of film release | Yes | to be transformed into `datetime` | 
| `'vote_count'` | ??? | Yes | Probably none | 
| `'vote_average'` | ??? | Yes | Probably none | 
| `'release_year'` | Year of film release | **No** | We take `'release_date'` instead| 
| `'budget_adj'` | Film's budget in USD (in 2018 prices) | Yes | Be aware of `0` values | 
| `'revenue_adj'` | Film's revenue in USD (in 2018 prices) | Yes | Be aware of `0` values | 

Unfortunatly, there are some encoding issues in the strings, I wasn't able to fix.

In [4]:
tmdb.id = tmdb.id.astype(str)

In [5]:
# Dropping duplicates and neglected features
tmdb.drop_duplicates(inplace=True)
tmdb.drop(axis=1, columns=['imdb_id', 'popularity', 'homepage', 'tagline', 'overview', 'release_year'], inplace=True)

In [6]:
# Calculating 'profit' as new derived feature
tmdb['profit'] = tmdb['revenue'] - tmdb['budget']
tmdb['profit_adj'] = tmdb['revenue_adj'] - tmdb['budget_adj']

In [7]:
# Reorganizing columns
tmdb = tmdb[
    [
        # Id and title
        'id', 'original_title', 
        # Rating data
        'vote_average', 'vote_count', 
        # Economical data
        'budget', 'revenue', 'profit', 
        # Adjusted economical data
        'budget_adj', 'revenue_adj', 'profit_adj',
        # Release- and run-time
        'release_date', 'runtime', 
        # Genre and content/keywords tags
        'genres', 'keywords', 
        #Companies and people involved
        'production_companies', 'director', 'cast'
    
    ]
]

In [8]:
# Creating 'datetime' format for 'release_date'
tmdb.release_date = pd.to_datetime(tmdb.release_date, infer_datetime_format=True)

In [9]:
# Caring for columns with list like entries

# Creating a function to create an expanded dataframe from columns with list-like entries for later analysis
def expand_listlike_column_with_ids_long(df, id_col='', list_col='', pattern=''):
    # Function to create an expanded dataframe from list like entries for later reuse
    expanded = df[[id_col, list_col]].copy()
    expanded[list_col] = df[list_col].str.split(pat=pattern)
    # Borrowed from 'https://mikulskibartosz.name/how-to-split-a-list-inside-a-dataframe-cell-into-rows-in-pandas-9849d8ff2401'
    expanded = expanded[list_col].apply(pd.Series) \
        .merge(expanded, right_index = True, left_index = True) \
        .drop([list_col], axis = 1) \
        .melt(id_vars = [id_col], value_name = list_col) \
        .drop("variable", axis = 1) \
        .dropna() \
        .drop_duplicates()
    return expanded

# Just to see...
expand_listlike_column_with_ids_long(tmdb, 'id', 'genres', '|').head(5)

,id,genres
0,135397,Action
1,76341,Action
2,262500,Adventure
3,140607,Action
4,168259,Action


In [22]:
def build_sets_from_list(list_col):
    # Customized function to make several trafos and lists for further analysis
        # 'id_with_list_stats' will return a long dictionary of all film ids and list items
        # 'list_stats' will return statistics (averages) of all list items with relation to average votes etc.
        # 'id_with_list_stats' will return statistics (averages) of all id with relation to list stats
    
    # Create basic expanded list
    ids_with_list_long = expand_listlike_column_with_ids_long(tmdb, 'id', list_col, '|')
    
    # Defined for further focus (main quantitative columns in tmdb)
    _additional_tmdb_info = [
        # focusses on 'vote_average', but could be expanded
        'id',
        'vote_average', #'vote_count', 
        #'budget', 'revenue', 'profit', 
        #'budget_adj', 'revenue_adj', 'profit_adj',
        #'release_date', 'runtime'
    ]
    
    # Intermediate list with info from tmdb
    _list = pd.merge(ids_with_list_long, tmdb[_additional_tmdb_info], on='id', how='left')

    # Statistics on list (with info from tmdb)
    list_stats = _list.groupby(list_col).mean()
    
    # All crammed into one big (intermediate) list
    _list_with_ids_and_stats = pd.merge(_list, list_stats, on=list_col, how='left', suffixes=('', '_' + list_col))
    
    # Stats from genres for ids
    id_with_list_stats = _list_with_ids_and_stats.groupby('id').mean().iloc[:, 1:]
    # if more stats are of interest, iloc needs to be changed
    
    return ids_with_list_long, list_stats, id_with_list_stats

ids_with_genres_long, genres_stats, ids_with_genres_stats = build_sets_from_list('genres')
ids_with_keywords_long, keywords_stats, ids_with_keywords_stats = build_sets_from_list('keywords')
ids_with_prodcos_long, prodcos_stats, ids_with_prodcos_stats = build_sets_from_list('production_companies')
ids_with_directors_long, directors_stats, ids_with_directors_stats = build_sets_from_list('director')
ids_with_actors_long, actors_stats, ids_with_actors_stats = build_sets_from_list('cast')

# Just to see... (1/2)
genres_stats.head(5)

,vote_average
genres,
Action,5.787752
Adventure,5.940585
Animation,6.403147
Comedy,5.905167
Crime,6.124889


In [24]:
# Just to see... (2/2)
ids_with_genres_stats.head(5)

,vote_average_genres
id,
100,6.015028
10001,5.785375
10002,6.111103
10003,5.837493
10004,5.812798


In [18]:
# Distribution of ratings (binnings necessary? Correlation to number of votes?)
# Analyze 'economics'
    # Are adjustments in budget, revenues and profits reasonable? ...Divide and correlate with year
    # Are (adjusted?) budgets, revenues and profits correlated to ratings?
# Analyze 'ingredients'
    # Which genres, keywords, prod_cos, directors, actors trigger the best ratings?
    # Are there 'dream combinations'? (e.g. average quality of actors, directors, prod_cos, genres, keywords correlated to ratings?)
# Answer 

In [27]:
def make_final_tmdb():
    tmdb1 = pd.merge(tmdb, ids_with_genres_stats, on='id', how='left')
    tmdb2 = pd.merge(tmdb1, ids_with_keywords_stats, on='id', how='left')
    tmdb3 = pd.merge(tmdb2, ids_with_prodcos_stats, on='id', how='left')
    tmdb4 = pd.merge(tmdb3, ids_with_directors_stats, on='id', how='left')
    tmdb_final = pd.merge(tmdb4, ids_with_actors_stats, on='id', how='left')
    return tmdb_final

tmbd_final = make_final_tmdb()

tmdb_final.head().T

,0,1,2,3,4
id,135397,76341,262500,140607,168259
original_title,Jurassic World,Mad Max: Fury Road,Insurgent,Star Wars: The Force Awakens,Furious 7
vote_average,6.5,7.1,6.3,7.5,7.3
vote_count,5562,6185,2480,5292,2947
budget,150000000,150000000,110000000,200000000,190000000
revenue,1513528810,378436354,295238201,2068178225,1506249360
profit,1363528810,228436354,185238201,1868178225,1316249360
budget_adj,1.38e+08,1.38e+08,1.012e+08,1.84e+08,1.748e+08
revenue_adj,1.39245e+09,3.48161e+08,2.71619e+08,1.90272e+09,1.38575e+09
profit_adj,1.25445e+09,2.10161e+08,1.70419e+08,1.71872e+09,1.21095e+09
